# Practice neural nets and ODEs

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
import time
import os
import logging

In [4]:
%matplotlib notebook
import matplotlib.pyplot as plt

### Configuration parameters

In [5]:
class objdict(dict):
    """ Wrap a dict so entries act as properties like an object.
        From https://goodcode.io/articles/python-dict-object/ .
    """
    
    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)

_args = {
    'network': 'odenet',          # 'resnet', 'odenet'
    'tol': 1e-3,
    'adjoint': False,
    'downsampling-method': 'res', # conv, res
    'nepochs': 160,
    'data_aug': True,
    'lr': 0.1,
    'batch_size': 128,
    'test_batch_size': 1000,
    'save': 'experiment1',
#    'debug': 'store_true',
    'gpu': 0
}

args = objdict(_args)

In [6]:
print(args)

{'network': 'odenet', 'tol': 0.001, 'adjoint': False, 'downsampling-method': 'res', 'nepochs': 160, 'data_aug': True, 'lr': 0.1, 'batch_size': 128, 'test_batch_size': 1000, 'save': 'experiment1', 'gpu': 0}


In [7]:
if args.adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint

### Helper functions

In [8]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def norm(dim):
    return nn.GroupNorm(min(32, dim), dim)

### Get data loaders

In [9]:
def get_mnist_loaders(data_aug=False, batch_size=128, test_batch_size=1000, perc=1.0):
    if data_aug:
        transform_train = transforms.Compose([
            transforms.RandomCrop(28, padding=4),
            transforms.ToTensor(),
        ])
    else:
        transform_train = transforms.Compose([
            transforms.ToTensor(),
        ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])

    train_loader = DataLoader(
        datasets.MNIST(root='./data/mnist', train=True, download=True, transform=transform_train), batch_size=batch_size,
        shuffle=True, num_workers=2, drop_last=True
    )

    train_eval_loader = DataLoader(
        datasets.MNIST(root='./data/mnist', train=True, download=True, transform=transform_test),
        batch_size=test_batch_size, shuffle=False, num_workers=2, drop_last=True
    )

    test_loader = DataLoader(
        datasets.MNIST(root='./data/mnist', train=False, download=True, transform=transform_test),
        batch_size=test_batch_size, shuffle=False, num_workers=2, drop_last=True
    )

    return train_loader, test_loader, train_eval_loader

def inf_generator(iterable):
    """Allows training with DataLoaders in a single infinite loop:
        for i, (x, y) in enumerate(inf_generator(train_loader)):
    """
    iterator = iterable.__iter__()
    while True:
        try:
            yield iterator.__next__()
        except StopIteration:
            iterator = iterable.__iter__()


def learning_rate_with_decay(batch_size, batch_denom, batches_per_epoch, boundary_epochs, decay_rates):
    initial_learning_rate = args.lr * batch_size / batch_denom

    boundaries = [int(batches_per_epoch * epoch) for epoch in boundary_epochs]
    vals = [initial_learning_rate * decay for decay in decay_rates]

    def learning_rate_fn(itr):
        lt = [itr < b for b in boundaries] + [True]
        i = np.argmax(lt)
        return vals[i]

    return learning_rate_fn


def one_hot(x, K):
    return np.array(x[:, None] == np.arange(K)[None, :], dtype=int)


def accuracy(model, dataset_loader):
    total_correct = 0
    for x, y in dataset_loader:
        x = x.to(device)
        y = one_hot(np.array(y.numpy()), 10)

        target_class = np.argmax(y, axis=1)
        predicted_class = np.argmax(model(x).cpu().detach().numpy(), axis=1)
        total_correct += np.sum(predicted_class == target_class)
    return total_correct / len(dataset_loader.dataset)


def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)


def get_logger(logpath, package_files=[], displaying=True, saving=True, debug=False):
    logger = logging.getLogger()
    if debug:
        level = logging.DEBUG
    else:
        level = logging.INFO
    logger.setLevel(level)
    if saving:
        info_file_handler = logging.FileHandler(logpath, mode="a")
        info_file_handler.setLevel(level)
        logger.addHandler(info_file_handler)
    if displaying:
        console_handler = logging.StreamHandler()
        console_handler.setLevel(level)
        logger.addHandler(console_handler)

    return logger


### ConcatConv2d for the ODENet NN

In [10]:
class ConcatConv2d(nn.Module):

    def __init__(self, dim_in, dim_out, ksize=3, stride=1, padding=0, dilation=1, groups=1, bias=True, transpose=False):
        super(ConcatConv2d, self).__init__()
        module = nn.ConvTranspose2d if transpose else nn.Conv2d
        self._layer = module(
            dim_in + 1, dim_out, kernel_size=ksize, stride=stride, padding=padding, dilation=dilation, groups=groups,
            bias=bias
        )

    def forward(self, t, x):
        tt = torch.ones_like(x[:, :1, :, :]) * t
        ttx = torch.cat([tt, x], 1)
        return self._layer(ttx)


### ODEFunc for the ODENet NN

In [11]:
class ODEfunc(nn.Module):

    def __init__(self, dim):
        super(ODEfunc, self).__init__()
        self.norm1 = norm(dim)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = ConcatConv2d(dim, dim, 3, 1, 1)
        self.norm2 = norm(dim)
        self.conv2 = ConcatConv2d(dim, dim, 3, 1, 1)
        self.norm3 = norm(dim)
        self.nfe = 0

    def forward(self, t, x):
        self.nfe += 1
        out = self.norm1(x)
        out = self.relu(out)
        out = self.conv1(t, out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(t, out)
        out = self.norm3(out)
        return out

### ODEBlock for the ODENet NN

In [12]:
class ODEBlock(nn.Module):

    def __init__(self, odefunc):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, 1]).float()

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        out = odeint(self.odefunc, x, self.integration_time, rtol=args.tol, atol=args.tol)
        return out[1]

    @property
    def nfe(self):
        return self.odefunc.nfe

    @nfe.setter
    def nfe(self, value):
        self.odefunc.nfe = value

In [13]:
class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        return x.view(-1, shape)

In [14]:
class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val

### ResBlock class. This is for the ResNet NN

In [15]:
class ResBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(ResBlock, self).__init__()
        self.norm1 = norm(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.norm2 = norm(planes)
        self.conv2 = conv3x3(planes, planes)

    def forward(self, x):
        shortcut = x

        out = self.relu(self.norm1(x))

        if self.downsample is not None:
            shortcut = self.downsample(out)

        out = self.conv1(out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(out)

        return out + shortcut


### Main function

In [16]:
is_odenet = args.network == 'odenet'

In [17]:
# define device that will execute the NN
device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

In [18]:
# downsampling layers
downsampling_layers = [
    nn.Conv2d(1, 64, 3, 1),
    ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
    ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
]

In [19]:
# feature layers
feature_layers = [ODEBlock(ODEfunc(64))] if is_odenet else [ResBlock(64, 64) for _ in range(6)]

In [20]:
# FC layers
fc_layers = [norm(64), nn.ReLU(inplace=True), nn.AdaptiveAvgPool2d((1, 1)), Flatten(), nn.Linear(64, 10)]

In [21]:
# Model
model = nn.Sequential(*downsampling_layers, *feature_layers, *fc_layers).to(device)

In [22]:
# criterion function
criterion = nn.CrossEntropyLoss().to(device)

In [23]:
# get data loaders
train_loader, test_loader, train_eval_loader = get_mnist_loaders(
    args.data_aug, args.batch_size, args.test_batch_size
)

In [24]:
# get data generator and batches per epoch
data_gen = inf_generator(train_loader)
batches_per_epoch = len(train_loader)

In [25]:
# get learning rate with decay
lr_fn = learning_rate_with_decay(
    args.batch_size, batch_denom=128, batches_per_epoch=batches_per_epoch, boundary_epochs=[60, 100, 140],
    decay_rates=[1, 0.1, 0.01, 0.001]
)

In [26]:
# get optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=0.9)

In [27]:
# initialize counters
best_acc = 0
batch_time_meter = RunningAverageMeter()
f_nfe_meter = RunningAverageMeter()
b_nfe_meter = RunningAverageMeter()
end = time.time()

In [28]:
# get logger
makedirs(args.save)
logger = get_logger(logpath=os.path.join(args.save, 'logs'))
logger.info(args)

device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

{'network': 'odenet', 'tol': 0.001, 'adjoint': False, 'downsampling-method': 'res', 'nepochs': 160, 'data_aug': True, 'lr': 0.1, 'batch_size': 128, 'test_batch_size': 1000, 'save': 'experiment1', 'gpu': 0}


In [29]:
# run
for itr in range(args.nepochs * batches_per_epoch):

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr_fn(itr)

    optimizer.zero_grad()
    x, y = data_gen.__next__()
    x = x.to(device)
    y = y.to(device)
    logits = model(x)
    loss = criterion(logits, y)

    if is_odenet:
        nfe_forward = feature_layers[0].nfe
        feature_layers[0].nfe = 0

    loss.backward()
    optimizer.step()

    if is_odenet:
        nfe_backward = feature_layers[0].nfe
        feature_layers[0].nfe = 0

    batch_time_meter.update(time.time() - end)
    if is_odenet:
        f_nfe_meter.update(nfe_forward)
        b_nfe_meter.update(nfe_backward)
    end = time.time()

    if itr % batches_per_epoch == 0:
        with torch.no_grad():
            train_acc = accuracy(model, train_eval_loader)
            val_acc = accuracy(model, test_loader)
            if val_acc > best_acc:
                torch.save({'state_dict': model.state_dict(), 'args': args}, os.path.join(args.save, 'model.pth'))
                best_acc = val_acc
            logger.info(
                "Epoch {:04d} | Time {:.3f} ({:.3f}) | NFE-F {:.1f} | NFE-B {:.1f} | "
                "Train Acc {:.4f} | Test Acc {:.4f}".format(
                    itr // batches_per_epoch, batch_time_meter.val, batch_time_meter.avg, f_nfe_meter.avg,
                    b_nfe_meter.avg, train_acc, val_acc
                )
            )

Epoch 0000 | Time 6.375 (6.375) | NFE-F 26.0 | NFE-B 0.0 | Train Acc 0.0992 | Test Acc 0.1009
Epoch 0001 | Time 4.656 (4.570) | NFE-F 20.2 | NFE-B 0.0 | Train Acc 0.9772 | Test Acc 0.9797
Epoch 0002 | Time 4.670 (4.516) | NFE-F 20.1 | NFE-B 0.0 | Train Acc 0.9897 | Test Acc 0.9905
Epoch 0003 | Time 4.669 (4.504) | NFE-F 20.1 | NFE-B 0.0 | Train Acc 0.9878 | Test Acc 0.9872
Epoch 0004 | Time 4.642 (4.503) | NFE-F 20.1 | NFE-B 0.0 | Train Acc 0.9914 | Test Acc 0.9904
Epoch 0005 | Time 4.661 (4.498) | NFE-F 20.1 | NFE-B 0.0 | Train Acc 0.9884 | Test Acc 0.9868
Epoch 0006 | Time 4.710 (4.521) | NFE-F 20.1 | NFE-B 0.0 | Train Acc 0.9931 | Test Acc 0.9912
Epoch 0007 | Time 4.663 (4.507) | NFE-F 20.1 | NFE-B 0.0 | Train Acc 0.9940 | Test Acc 0.9927
Epoch 0008 | Time 4.682 (4.528) | NFE-F 20.1 | NFE-B 0.0 | Train Acc 0.9953 | Test Acc 0.9936
Epoch 0009 | Time 4.797 (4.597) | NFE-F 20.1 | NFE-B 0.0 | Train Acc 0.9954 | Test Acc 0.9937


KeyboardInterrupt: 